# Capstone 3: Data Wrangling
## By: Pedro Rodriguez

I will import the Listing and calendar data obtained from Kaggle to see what features have and determine what elements are needed to know how the cleaning fee impacts the monthly booking. Before we start cleaning the data, we have to determine what features are necessary to decide how the cleaning fee impacts the monthly availability of Airbnb.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

airbnb = pd.read_csv('/Users/pedrorodriguez/Desktop/Springboard/Capstone_3/Raw Data/listings.csv')
airbnb_cal = pd.read_csv('/Users/pedrorodriguez/Desktop/Springboard/Capstone_3/Raw Data/calendar.csv')

airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   listing_url                       3818 non-null   object 
 2   scrape_id                         3818 non-null   int64  
 3   last_scraped                      3818 non-null   object 
 4   name                              3818 non-null   object 
 5   summary                           3641 non-null   object 
 6   space                             3249 non-null   object 
 7   description                       3818 non-null   object 
 8   experiences_offered               3818 non-null   object 
 9   neighborhood_overview             2786 non-null   object 
 10  notes                             2212 non-null   object 
 11  transit                           2884 non-null   object 
 12  thumbn

In [2]:
airbnb.head(3)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15


In [3]:
airbnb['room_type'].unique()

array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object)

### Let's identify which features are necessary to determine how the cleaning fee impacts the monthly booking, and then create a new DataFrame. 

In [4]:
df = airbnb[['id', 'zipcode', 'latitude','longitude', 'property_type', 'cancellation_policy',
             'room_type','price', 'cleaning_fee', 'availability_30',
             'review_scores_rating', 'review_scores_cleanliness']]
df.head()

,id,zipcode,latitude,longitude,property_type,cancellation_policy,room_type,price,cleaning_fee,availability_30,review_scores_rating,review_scores_cleanliness
0,241032,98119,47.636289,-122.371025,Apartment,moderate,Entire home/apt,$85.00,NaN,14,95.0,10.0
1,953595,98119,47.639123,-122.365666,Apartment,strict,Entire home/apt,$150.00,$40.00,13,96.0,10.0
2,3308979,98119,47.629724,-122.369483,House,strict,Entire home/apt,$975.00,$300.00,1,97.0,10.0
3,7421966,98119,47.638473,-122.369279,Apartment,flexible,Entire home/apt,$100.00,NaN,0,NaN,NaN
4,278830,98119,47.632918,-122.372471,House,strict,Entire home/apt,$450.00,$125.00,30,92.0,9.0


## Let's start cleaning the new DataFrame. 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3818 non-null   int64  
 1   zipcode                    3811 non-null   object 
 2   latitude                   3818 non-null   float64
 3   longitude                  3818 non-null   float64
 4   property_type              3817 non-null   object 
 5   cancellation_policy        3818 non-null   object 
 6   room_type                  3818 non-null   object 
 7   price                      3818 non-null   object 
 8   cleaning_fee               2788 non-null   object 
 9   availability_30            3818 non-null   int64  
 10  review_scores_rating       3171 non-null   float64
 11  review_scores_cleanliness  3165 non-null   float64
dtypes: float64(4), int64(2), object(6)
memory usage: 358.1+ KB


The datasets seem to have missing values in the Cleaning fee because it is 0; the host doesn't price for cleaning. Let's start cleaning this feature by deleting the dollar sign and putting it as a float type for feature analysis. 

In [6]:
df = df.fillna(0)
df['cleaning_fee'] = df['cleaning_fee'].str.replace('$', '').fillna(0).astype(float)
df['cleaning_fee'].unique()

array([  0.,  40., 300., 125.,  25.,  15., 150.,  95.,  85.,  89.,  35.,
       250., 200.,  65., 100.,  80.,  99.,  50.,  20.,  55.,  75.,  30.,
        60., 120.,  78.,  12.,  45.,  10., 264., 180.,  90.,   7., 131.,
         8.,   5., 185., 199., 175., 110., 155., 111.,  72., 105., 160.,
        13., 275.,  28.,  70., 209.,  82., 195., 145.,  22., 225., 169.,
       119.,  29., 140.,  61.,  49., 108.,   6.,  26.,  83.,  18.,  19.,
       117., 112.,  58.,  16., 170.,  64., 113.,  79., 130.,  96., 149.,
       164., 159.,  32., 184., 109., 107., 274., 143.,  88., 229.,  38.,
        69., 135.,  59., 101.,  67., 240., 137., 134.,  21., 189.,   9.,
        17., 106.,  24., 165.,  39.,  68.,  27.,  87.,  42.,  71., 194.,
       129., 210., 178.,  76.,  97., 179.,  52., 142., 230.])

In [7]:
df.head()

,id,zipcode,latitude,longitude,property_type,cancellation_policy,room_type,price,cleaning_fee,availability_30,review_scores_rating,review_scores_cleanliness
0,241032,98119,47.636289,-122.371025,Apartment,moderate,Entire home/apt,$85.00,0.0,14,95.0,10.0
1,953595,98119,47.639123,-122.365666,Apartment,strict,Entire home/apt,$150.00,40.0,13,96.0,10.0
2,3308979,98119,47.629724,-122.369483,House,strict,Entire home/apt,$975.00,300.0,1,97.0,10.0
3,7421966,98119,47.638473,-122.369279,Apartment,flexible,Entire home/apt,$100.00,0.0,0,0.0,0.0
4,278830,98119,47.632918,-122.372471,House,strict,Entire home/apt,$450.00,125.0,30,92.0,9.0


## Let's now clean the price feature. 

In [8]:
df['price'] = df['price'].str.replace('$', '').fillna(0)
df['price'] = df['price'].str.replace(',', '').astype(float)
df.head()

,id,zipcode,latitude,longitude,property_type,cancellation_policy,room_type,price,cleaning_fee,availability_30,review_scores_rating,review_scores_cleanliness
0,241032,98119,47.636289,-122.371025,Apartment,moderate,Entire home/apt,85.0,0.0,14,95.0,10.0
1,953595,98119,47.639123,-122.365666,Apartment,strict,Entire home/apt,150.0,40.0,13,96.0,10.0
2,3308979,98119,47.629724,-122.369483,House,strict,Entire home/apt,975.0,300.0,1,97.0,10.0
3,7421966,98119,47.638473,-122.369279,Apartment,flexible,Entire home/apt,100.0,0.0,0,0.0,0.0
4,278830,98119,47.632918,-122.372471,House,strict,Entire home/apt,450.0,125.0,30,92.0,9.0


## Let's now clean the zip code feature. 

In [9]:
df['zipcode'].unique()

array(['98119', '98109', '98107', '98117', 0, '98103', '98105', '98115',
       '98101', '98122', '98112', '98144', '99\n98122', '98121', '98102',
       '98199', '98104', '98134', '98136', '98126', '98146', '98116',
       '98177', '98118', '98108', '98133', '98106', '98178', '98125'],
      dtype=object)

In [10]:
df['zipcode'] = df['zipcode'].str.replace('99\n98122', '98122').fillna(0)
df['zipcode'] = df['zipcode'].astype(int)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         3818 non-null   int64  
 1   zipcode                    3818 non-null   int64  
 2   latitude                   3818 non-null   float64
 3   longitude                  3818 non-null   float64
 4   property_type              3818 non-null   object 
 5   cancellation_policy        3818 non-null   object 
 6   room_type                  3818 non-null   object 
 7   price                      3818 non-null   float64
 8   cleaning_fee               3818 non-null   float64
 9   availability_30            3818 non-null   int64  
 10  review_scores_rating       3818 non-null   float64
 11  review_scores_cleanliness  3818 non-null   float64
dtypes: float64(6), int64(3), object(3)
memory usage: 358.1+ KB


In [12]:
df.to_csv('/Users/pedrorodriguez/Desktop/Springboard/Capstone_3/Raw Data/Airbnb_EDA.csv')

## Calendar data
I will use the calendar data to understand how the market behaves throughout the year. Try to identify high and low seasons for Airbnb.

In [13]:
airbnb_cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   listing_id  1393570 non-null  int64 
 1   date        1393570 non-null  object
 2   available   1393570 non-null  object
 3   price       934542 non-null   object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB


### Let's clean the price column by eliminating the dollar symbol and change the data type to float.

In [14]:
airbnb_cal['price'] = airbnb_cal['price'].str.replace('$', '').fillna('0')
airbnb_cal['price'] = airbnb_cal['price'].str.replace(',', '').astype(float)

In [15]:
airbnb_cal.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,85.0
1,241032,2016-01-05,t,85.0
2,241032,2016-01-06,f,0.0
3,241032,2016-01-07,f,0.0
4,241032,2016-01-08,f,0.0


In [16]:
airbnb_cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   listing_id  1393570 non-null  int64  
 1   date        1393570 non-null  object 
 2   available   1393570 non-null  object 
 3   price       1393570 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 42.5+ MB


### The date is in object format so that I will change to DateTime format, and the available data is in t= true and f= false. I'm going to change true= 1 and false=0.

In [17]:
airbnb_cal['date'] = pd.to_datetime(airbnb_cal['date'])
airbnb_cal['available'] = airbnb_cal['available'].replace({'t': '1', 'f': '0'}).astype(int)

In [18]:
airbnb_cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   listing_id  1393570 non-null  int64         
 1   date        1393570 non-null  datetime64[ns]
 2   available   1393570 non-null  int64         
 3   price       1393570 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 42.5 MB


In [20]:
airbnb_cal.to_csv('/Users/pedrorodriguez/Desktop/Springboard/Capstone_3/Raw Data/Calendar_EDA.csv')

## Summary

When we open the DataFrame from Kaggle, we notice missing values, incorrect data type, and excess information. Before we start cleaning the data, we have to determine what features are necessary to decide how the cleaning fee impacts the monthly availability of Airbnb. We keep zip code, latitude, and longitude for visualization purposes—property type and price to understand how the prices change per home type. And the cleaning fee, availability, and rating score to determine how the cleaning fee affects the booking frequency. 
After creating a new DataFrame with the necessary data, we cleaned the price and cleaning fee data by fill nulls values with 0, eliminate the dollar symbols, and change from object to float type for further analysis. The date is in object format so that I will change to DateTime format, and the available data is in t= true and f= false. I'm going to change true= 1 and false=0 and fill the price with 0.
